In [19]:
#IMPORT LIBRARIES
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as cx
import matplotlib.patheffects as pe

In [20]:
# SETTING VARIABLES

#csv containing dataframe
my_csv_file = " "
#name of column that is going to be plotted
var_name = ""
#name of attribbute of said column that is going to be plotted
atrib_name = ""
#title for the plot
title = ""

In [21]:
# RREATE DF FROM CSV AND SHAPEFILE WITH DISTRICTS

#read the csv file
df = pd.read_csv(my_csv_file, sep = ";")

# change lat and long format
df.Latitud.replace(to_replace=",",value=".", inplace=True, regex=True)

df.Longitud.replace(to_replace=",", value=".", inplace=True, regex=True)

#drop cases where there is no localization data
df = df[~df.Latitud.str.contains(" ")]

df = df[~df.Longitud.str.contains(" ")]


#Transform df into a geodataframe
dfgeo = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitud, df.Latitud))

#Set crs of coordinates in dfgeo to match distritos
dfgeo = dfgeo.set_crs("EPSG:4326")

#Create geodataframe with distritos 
distritos = gpd.read_file("DISTRITOS-polygon.shp")
dfgeo = dfgeo.set_crs("EPSG:4326")

#Join both dataframes according to in wich district is located each case
geojoined = gpd.sjoin(dfgeo, distritos, op="within")

c:\Users\patricio\anaconda3\lib\site-packages\geopandas\array.py:275: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return GeometryArray(vectorized.points_from_xy(x, y, z), crs=crs)


In [ ]:
#DEFINE FUNCTION TO CREATE A LIST WITH PERCENTAGES OF ATTRIBUTE IN EACH DISTRICT

list_porc = []

def porcentaje(distri):
    geodist=geojoined[geojoined.Name == distri]

    porcdist = geodist[var_name][geodist[var_name] == atrib_name].count() *100 / geodist[var_name].count()

    list_porc.append(porcdist)

    return list_porc

dist= distritos.Name.values

list_porc=[]

for i in dist:
    porcentaje(i)


#ADD LIST AS A COLUMN IN DISTRICTS DATAFRAME

distritos["new_var"]=list_porc
distritos["new_var"]=distritos["new_var"].round(decimals=1)

list_porc

In [ ]:
#PLOT THE MAP 
fig, ax = plt.subplots(figsize=(8, 8))

distritos.plot(ax=ax,
                column="new_var",
                cmap="Blues",
                legend=False,
                edgecolor="0")


distritos.apply(lambda x: ax.annotate(text=x['new_var'],
                 xy=x.geometry.centroid.coords[0],
                 ha='center',
                 fontsize=13,
                 color="white",
                 path_effects=[pe.withStroke(linewidth=3, foreground="black")]),
                 axis=1)

ax.axis("off")

ax.set_title(title, fontsize=18, font="monospace", ha="center")

